## Week 2 Day 1

And now! Our first look at OpenAI Agents SDK

You won't believe how lightweight this is..

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">The OpenAI Agents SDK Docs</h2>
            <span style="color:#00bfff;">The documentation on OpenAI Agents SDK is really clear and simple: <a href="https://openai.github.io/openai-agents-python/">https://openai.github.io/openai-agents-python/</a> and it's well worth a look.
            </span>
        </td>
    </tr>
</table>

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
import gradio as gr




In [2]:
# The usual starting point

load_dotenv(override=True)


True

In [3]:

# Make an agent with name, instructions, model

agent = Agent(name="Jokester", instructions="You are a joke teller", model="gpt-4o-mini")

In [4]:
# Run the joke with Runner.run(agent, prompt) then print final_output

async def ask_questions(message):
    with trace("Telling a joke"):
        result = await Runner.run(agent, message)
        return result.final_output  


In [6]:
demo = gr.Interface(
    fn=ask_questions,
    inputs=gr.Textbox(label="Enter message"),
    outputs=gr.Textbox(label="Response"),
)

demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


In [7]:
import gradio as gr
import asyncio

# Your async function
async def ask_questions(message):
    with trace("Telling a joke"):
        result = await Runner.run(agent, message)
        return result.final_output

chat_history = []


# Wrapper to call async from sync Gradio interface
def gradio_interface(message, history):
    response = asyncio.run(ask_questions(message))
    chat_history.append({"user": message, "agent": response})

    history += f"\nUser: {message}\nAgent: {response} \n"
    return response, history

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Async Chat Interface")

    msg_input = gr.Textbox(label="Your Message")
    btn = gr.Button("Send")

    output = gr.Textbox(label="Agent's Response")
    
    history_box = gr.Textbox(label="Chat History", lines=10, interactive=False)


    btn.click(
        fn=gradio_interface,
        inputs=[msg_input, history_box],
        outputs=[output, history_box]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7894
* To create a public link, set `share=True` in `launch()`.


In [57]:
import gradio as gr
import asyncio

# Your async function
async def ask_questions(message):
    with trace("Telling a joke"):
        result = await Runner.run(agent, message)
        return result.final_output

chat_history = []


# Wrapper to call async from sync Gradio interface
def gradio_interface(message):
    response = asyncio.run(ask_questions(message))
    chat_history.append({"user": message, "agent": response})
    formatted = ""
    for turn in chat_history:
        formatted += f"User: {turn['user']}\nAgent: {turn['agent']}\n\n"
 
    return response, formatted

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Async Chat Interface")

    msg_input = gr.Textbox(label="Your Message")
    btn = gr.Button("Send")

    output = gr.Textbox(label="Agent's Response")
    
    history_box = gr.Textbox(label="Chat History", lines=10, interactive=False)


    btn.click(
        fn=gradio_interface,
        inputs=[msg_input],
        outputs=[output, history_box]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


In [9]:
import gradio as gr
import asyncio
import time
from typing import List, Dict

# Your async function
async def ask_questions(message: str) -> str:
    with trace("Telling a joke"):
        result = await Runner.run(agent, message)
        return result.final_output

# Global chat history
chat_history: List[Dict[str, str]] = []

def gradio_interface(message: str, history: List[List[str]]) -> tuple:
    """
    Enhanced Gradio interface function with better error handling and formatting
    """
    if not message.strip():
        return history, history, ""
    
    try:
        # Run the async function
        response = asyncio.run(ask_questions(message))
        
        # Update chat history
        chat_history.append({"user": message, "agent": response, "timestamp": time.time()})
        
        # Update Gradio chat history format
        history.append([message, response])
        
        return history, history, ""  # Clear input
        
    except Exception as e:
        error_msg = f"Error: {str(e)}"
        history.append([message, error_msg])
        return history, history, ""

def clear_history():
    """Clear the chat history"""
    global chat_history
    chat_history = []
    return [], []

def export_history() -> str:
    """Export chat history as formatted text"""
    if not chat_history:
        return "No chat history to export."
    
    formatted = "=== Chat History Export ===\n\n"
    for i, turn in enumerate(chat_history, 1):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(turn["timestamp"]))
        formatted += f"Turn {i} ({timestamp}):\n"
        formatted += f"User: {turn['user']}\n"
        formatted += f"Agent: {turn['agent']}\n"
        formatted += "-" * 50 + "\n\n"
    
    return formatted

# Enhanced Gradio UI
with gr.Blocks(title="Async Chat Interface", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🤖 Async Chat Interface
    
    Chat with your async agent in real-time. Messages are processed asynchronously for better performance.
    """)
    
    with gr.Row():
        with gr.Column(scale=3):
            # Main chat interface using Gradio's built-in chatbot component
            chatbot = gr.Chatbot(
                label="Chat with Agent",
                height=400,
                show_label=True,
                avatar_images=["👤", "🤖"]
            )
            
            with gr.Row():
                msg_input = gr.Textbox(
                    label="Your Message",
                    placeholder="Type your message here...",
                    lines=2,
                    scale=4
                )
                send_btn = gr.Button("Send 📤", variant="primary", scale=1)
            
            with gr.Row():
                clear_btn = gr.Button("Clear Chat 🗑️", variant="secondary")
                export_btn = gr.Button("Export History 📋", variant="secondary")
        
        with gr.Column(scale=1):
            gr.Markdown("### Chat Controls")
            
            stats_display = gr.Markdown("**Messages sent:** 0")
            
            # Export area
            export_output = gr.Textbox(
                label="Exported History",
                lines=10,
                interactive=False,
                visible=False
            )
    
    # Event handlers
    def update_stats():
        return f"**Messages sent:** {len(chat_history)}"
    
    def send_message(message, history):
        result = gradio_interface(message, history)
        return result + (update_stats(),)
    
    def clear_and_update():
        clear_history()
        return [], [], update_stats()
    
    def export_and_show():
        exported = export_history()
        return exported, gr.update(visible=True)
    
    # Button clicks
    send_btn.click(
        fn=send_message,
        inputs=[msg_input, chatbot],
        outputs=[chatbot, chatbot, msg_input, stats_display]
    )
    
    # Enter key support
    msg_input.submit(
        fn=send_message,
        inputs=[msg_input, chatbot],
        outputs=[chatbot, chatbot, msg_input, stats_display]
    )
    
    clear_btn.click(
        fn=clear_and_update,
        outputs=[chatbot, chatbot, stats_display]
    )
    
    export_btn.click(
        fn=export_and_show,
        outputs=[export_output, export_output]
    )

    demo.launch()



/var/folders/z9/yt34ddps7xx4bv4m4xb674g00000gn/T/ipykernel_6881/3507197931.py:71: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


* Running on local URL:  http://127.0.0.1:7896
* To create a public link, set `share=True` in `launch()`.


## Now go and look at the trace

https://platform.openai.com/traces

mport numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(), "image")
demo.launch()